# LSTM autoencoder in torch

A reconstruction LSTM autoencoder in pytorch is listed below. The original Keras model and its explanation can be found in a blog by Jason Brownlee at 
https://machinelearningmastery.com/lstm-autoencoders/

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim=100):
        super().__init__()
        n_layers = 1
        # n_layers, batch_size, hidden_dim)
        self.lstm_1 = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)
        self.lstm_2 = nn.LSTM(hidden_dim, hidden_dim, n_layers, batch_first=True)
        self.linear = nn.Linear(100, 1)
        
    def forward(self, x):
        seq_len = x.size()[1]
        # encoding
        # [batch_size, seq_len, input_dim]
        x, (hidden_state, cell_state) = self.lstm_1(x)
        # [batch_size, seq_len, hidden_dim]
        
        # decoding
        x = F.relu(x[:, -1, :])
        # [batch_size, hidden_dim]
        x = x.unsqueeze(1).repeat(1, seq_len, 1)
        # [batch_size, seq_len, hidden_dim]
        x, (hidden_state, cell_state) = self.lstm_2(x)
        # [batch_size, seq_len, hidden_dim]
        x = F.relu(x)
        x = torch.cat([
            self.linear(x[:, i, :]).unsqueeze(1) for i in range(seq_len)],
            1)
        # [batch_size, seq_len, 1]
        return x

In [3]:
from tqdm import tqdm

epochs = 300
seq_len = 9
seq = np.linspace(0.1, 0.9, seq_len).reshape((1, -1, 1)).astype(np.float32)

model = LSTMAutoencoder(1).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(),
)

inp = torch.from_numpy(seq).to(device)
pbar = tqdm(range(epochs))
for epoch in pbar:
    model.train()
    optimizer.zero_grad()
    output = model(inp)
    loss = criterion(output, inp)
    loss.backward()
    optimizer.step()
    pbar.set_description(f'loss: {loss.item():.4f}')

loss: 0.0000: 100%|██████████| 300/300 [00:01<00:00, 191.95it/s]


In [4]:
model.eval()
yhat = model(inp)
yhat[0, :, 0]

tensor([0.1081, 0.1970, 0.2959, 0.3959, 0.4984, 0.6028, 0.7061, 0.8044, 0.8923],
       device='cuda:0', grad_fn=<SelectBackward>)